# Result Plots

## Medium Dataset

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

medium_gt_trained = pd.read_csv('../data/test_results_medium/test_output_gt.csv')
medium_vio_trained = pd.read_csv('../data/test_results_medium/test_output_vio.csv')

hard_gt_trained = pd.read_csv('../data/test_results_hard/test_output_gt.csv')
hard_vio_trained = pd.read_csv('../data/test_results_hard/test_output_vio.csv')

In [ ]:
def to_float(x):
    if isinstance(x, str):
        # Handle string like "tensor(0.5041)"
        return float(x.replace("tensor(", "").replace(")", ""))
    elif isinstance(x, torch.Tensor):
        return x.item()
    elif isinstance(x, float) or isinstance(x, int):
        return float(x)
    else:
        raise TypeError(f"Unrecognized type in tensor column: {type(x)}")

def clean_df(df: pd.DataFrame):
    df["Model vs. VIO"] = df.iloc[:, 1].apply(to_float)
    df["VIO vs. GT"] = df.iloc[:, 2].apply(to_float)
    df["Model vs. GT"] = df.iloc[:, 3].apply(to_float)

clean_df(medium_gt_trained)
clean_df(medium_vio_trained)
clean_df(hard_gt_trained)
clean_df(hard_vio_trained)
medium_gt_trained.head(5)

KeyError: 1

,Unnamed: 0,Model vs. VIO Pose,VIO vs. Ground Truth Pose,Model vs. Ground Truth Pose
0,0,tensor(0.5041),tensor(1.0234),tensor(0.5414)
1,1,tensor(0.5043),tensor(1.0236),tensor(0.5412)
2,2,tensor(0.5042),tensor(1.0237),tensor(0.5413)
3,3,tensor(0.5042),tensor(1.0238),tensor(0.5414)
4,4,tensor(0.5042),tensor(1.0238),tensor(0.5413)


In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
from typing import Sequence

def plot_time_series(timestamps: Sequence, model_vio, vio_gt, model_gt):
    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, model_vio, label="Model vs. VIO")
    plt.plot(timestamps, vio_gt, label="VIO vs. Ground Truth")
    plt.plot(timestamps, model_gt, label="Model vs. Ground Truth")
    plt.xlabel("Timestamp")
    plt.ylabel("Pose Error")
    plt.title("Error Over Time")
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_time_lagged_correlation(timestamps: Sequence, model_vio, vio_gt, max_lag: int = 20):
    df = pd.DataFrame({
        "timestamp": timestamps,
        "model_vio": model_vio,
        "vio_gt": vio_gt
    }).set_index("timestamp")

    lags = range(-max_lag, max_lag + 1)
    corrs = [df["model_vio"].corr(df["vio_gt"].shift(lag)) for lag in lags]

    plt.figure()
    plt.plot(lags, corrs, marker='o')
    plt.xlabel("Lag (samples)")
    plt.ylabel("Correlation")
    plt.title("Lagged Correlation: Model-VIO vs VIO-GT")
    plt.axhline(y=0, color="gray", linestyle="--")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_scatter_colored_by_time(model_vio, vio_gt, timestamps: Sequence):
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(model_vio, vio_gt, c=timestamps, cmap="viridis")
    plt.xlabel("Model vs. VIO Error")
    plt.ylabel("VIO vs. Ground Truth Error")
    plt.title("Scatter Colored by Timestamp")
    plt.colorbar(scatter, label="Timestamp")
    plt.tight_layout()
    plt.show()

def plot_rolling_correlation(timestamps: Sequence, model_vio, vio_gt, window: int = 50):
    df = pd.DataFrame({
        "timestamp": timestamps,
        "model_vio": model_vio,
        "vio_gt": vio_gt
    }).set_index("timestamp")

    rolling_corr = df["model_vio"].rolling(window=window).corr(df["vio_gt"])

    plt.figure()
    plt.plot(rolling_corr.index, rolling_corr, label=f"Rolling Corr (window={window})")
    plt.xlabel("Timestamp")
    plt.ylabel("Correlation")
    plt.title("Rolling Correlation: Model-VIO vs VIO-GT")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_all(df: pd.DataFrame):
    timestamps = df.index if df.index.is_monotonic_increasing else range(len(df))
    plot_time_series(timestamps, df["Model vs. VIO"], df["VIO vs. GT"], df["Model vs. GT"])
    plot_time_lagged_correlation(timestamps, df["Model vs. VIO"], df["VIO vs. GT"])
    plot_scatter_colored_by_time(df["Model vs. VIO"], df["VIO vs. GT"], timestamps)
    plot_rolling_correlation(timestamps, df["Model vs. VIO"], df["VIO vs. GT"])
    
    # Reports the overall correlation
    print(df["Model vs. GT"].corr(df["VIO vs. GT"]))
